# 実行環境の作成

## python3.9

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

In [2]:
!python --version

Python 3.9.12


## Google Cloud Storage

下記コードでGCPに接続

In [3]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [5]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/15 12:50:14.532001 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/15 12:50:14.546255 Opening GCS connection...
2022/09/15 12:50:15.938109 Mounting file system "statistics-hyogo"...
2022/09/15 12:50:15.938540 File system has been successfully mounted.


## ESTAT_APP_ID

In [6]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [7]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

## 統計表のメタ情報を取得

'statsDataId'に統計表コードを指定

In [8]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [9]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
print(meta)

{'GET_META_INFO': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-15T21:50:32.408+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'DATA_FORMAT': 'J'}, 'METADATA_INF': {'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'COLLECT_AREA': '全国', 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 500976, 'UPDATED_DATE': '2022-03-04', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '都道府県データ', 'TABULATION_SUB_CATEGORY1': '基礎データ'}, 'DESCRIPTION': {'TABULATION_CATEGORY_EXPLANATION': '社会・人口統計体系の都道府県ごとに集計したデータを提供します。'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLASS_INF': {'CLASS_OBJ': [{'@id': 'tab', '@name': '観測値', 'CLASS': {'@code': '00001', 

## 統計表csvを作成

カテゴリコード'cat01'のリストを作成する関数

In [18]:
def generate_cards(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']
  print(STAT_NAME)

  TITLE = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['TITLE']['$']
  print(TITLE)

  # カテゴリ一覧
  categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']
  print(categories)

  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

cards = generate_cards(statsDataId)
print(cards)

社会・人口統計体系
Ａ　人口・世帯
[{'@code': 'A1101', '@name': 'A1101_総人口', '@level': '1', '@unit': '人'}, {'@code': 'A110101', '@name': 'A110101_総人口（男）', '@level': '1', '@unit': '人'}, {'@code': 'A110102', '@name': 'A110102_総人口（女）', '@level': '1', '@unit': '人'}, {'@code': 'A1102', '@name': 'A1102_日本人人口', '@level': '1', '@unit': '人'}, {'@code': 'A110201', '@name': 'A110201_日本人人口（男）', '@level': '1', '@unit': '人'}, {'@code': 'A110202', '@name': 'A110202_日本人人口（女）', '@level': '1', '@unit': '人'}, {'@code': 'A1201', '@name': 'A1201_0～4歳人口', '@level': '1', '@unit': '人'}, {'@code': 'A120101', '@name': 'A120101_0～4歳人口（男）', '@level': '1', '@unit': '人'}, {'@code': 'A120102', '@name': 'A120102_0～4歳人口（女）', '@level': '1', '@unit': '人'}, {'@code': 'A1202', '@name': 'A1202_5～9歳人口', '@level': '1', '@unit': '人'}, {'@code': 'A120201', '@name': 'A120201_5～9歳人口（男）', '@level': '1', '@unit': '人'}, {'@code': 'A120202', '@name': 'A120202_5～9歳人口（女）', '@level': '1', '@unit': '人'}, {'@code': 'A1203', '@name': 'A1203_10～14歳人口', '@l